In [ ]:
#Looking at Isoform Weight and Length of one CSV File. 

In [16]:
import pandas as pd

# read the CSV file into a pandas DataFrame
df = pd.read_csv('/Users/anabelyong/miniconda3/lib/python3.10/site-packages/jupyterlab/Subset750RawfilesCut/CSV_Files_No_Separation/SeparateProteinIDs_No_Separation_ProteinGroups.output/Unmerged_CSVs/SeparateProteinIDs_Picked_protein_group_no_remap_4000Files_withScore_unmerged.csv')

# filter for rows where the 'Protein Uniprot' column matches the desired pattern
df = df[df['Protein Uniprot'].str.match(r'(\w+-\d+)')]
print(df)
# save the filtered DataFrame to a new CSV file
df.to_csv('/Users/anabelyong/miniconda3/lib/python3.10/site-packages/jupyterlab/Subset750RawfilesCut/CSV_Files_No_Separation/SeparateProteinIDs_No_Separation_ProteinGroups.output/Uniprot_IDs_Isoforms_Raw_Files/Uniprot_IDs_isoforms_Picked_protein_group_no_remap_4000Files.csv', index=False)

        Q-value Protein Type Protein Uniprot  \
0      0.000327           sp        P30044-2   
1      0.000327           sp        Q8WTV0-4   
2      0.000327           sp        O00154-4   
3      0.000327           sp        Q02880-2   
4      0.000327           sp        P35658-4   
...         ...          ...             ...   
12593  0.507363      REV__sp        Q8NI60-2   
12594  0.507363      REV__sp        Q9NSC5-2   
12595  0.507363           sp        Q9Y6A2-3   
12597  0.507407      REV__sp        Q6ISS4-2   
12598  0.507407      REV__sp        Q8N884-2   

                                           Protein Label  Protein names  \
0      PRDX5_HUMAN;sp|P30044-3|PRDX5_HUMAN;sp|P30044-...            NaN   
1      SCRB1_HUMAN;sp|Q8WTV0-2|SCRB1_HUMAN;sp|Q8WTV0-...            NaN   
2      BACH_HUMAN;sp|O00154-2|BACH_HUMAN;sp|O00154-3|...            NaN   
3                      TOP2B_HUMAN;sp|Q02880|TOP2B_HUMAN            NaN   
4      NU214_HUMAN;sp|P35658-3|NU214_HUMAN;sp|P3

# Doing above code for all the CSV Files which show unmerged proteins (Isoforms or RiboSeq data)

In [17]:
import os
import pandas as pd

# specify the directory containing the input CSV files
input_dir = '/Users/anabelyong/miniconda3/lib/python3.10/site-packages/jupyterlab/Subset750RawfilesCut/CSV_Files_No_Separation/SeparateProteinIDs_No_Separation_ProteinGroups.output/Unmerged_CSVs/'

# specify the directory where filtered CSV files will be saved
output_dir = '/Users/anabelyong/miniconda3/lib/python3.10/site-packages/jupyterlab/Subset750RawfilesCut/CSV_Files_No_Separation/SeparateProteinIDs_No_Separation_ProteinGroups.output/Uniprot_IDs_Isoforms_Raw_Files/'

# iterate over all files in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        # read the CSV file into a pandas DataFrame
        df = pd.read_csv(os.path.join(input_dir, filename))

        # filter for rows where the 'Protein Uniprot' column matches the desired pattern
        df = df[df['Protein Uniprot'].str.match(r'(\w+-\d+)')]

        # save the filtered DataFrame to a new CSV file in the output directory
        output_filename = os.path.join(output_dir, f'filtered_{filename}')
        df.to_csv(output_filename, index=False)


# EXAMPLE for one csv file ({n} Uniprot IDs)
#Print the above isoforms, molecular weight and number of amino acids {of isoforms} by individual raw files. 

In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep

def get_fasta_isoform(isoform_id):
    # define the UniProt API endpoint for the protein isoform
    url = f"https://www.uniprot.org/uniprot/{isoform_id}.fasta"

    # send a GET request to the UniProt API endpoint to retrieve the protein sequence
    response = requests.get(url)

    lines = response.text.split('\n')
    fasta_isoform = ''.join(lines[1:])

    return fasta_isoform

# Load the UniProt IDs from a CSV file
df = pd.read_csv('/Users/anabelyong/miniconda3/lib/python3.10/site-packages/jupyterlab/Subset750RawfilesCut/CSV_Files_No_Separation/SeparateProteinIDs_No_Separation_ProteinGroups.output/Uniprot_IDs_Isoforms_Raw_Files/filtered_SeparateProteinIDs_Picked_protein_group_no_remap_100Files_withScore_unmerged.csv', nrows=100)

mass_df = pd.DataFrame(columns=['Uniprot ID', 'Molecular Weight'])
length_df = pd.DataFrame(columns=['Uniprot ID', 'Number of amino acids'])

for uniprot_id in df['Protein Uniprot']:
    try:
        # Define the URL for the ProtParam tool
        url = "https://web.expasy.org/cgi-bin/protparam/protparam"

        # Define the payload for the GET request
        sequence = get_fasta_isoform(uniprot_id)
        payload = {"sequence": sequence}

        # Submit the GET request and get the response
        response = requests.get(url, params=payload)

        soup = BeautifulSoup(response.content, "html.parser")
        num_amino_acids_raw = soup.find(string="Number of amino acids:").next
        mol_weight_raw = soup.find(string="Molecular weight:").next

        num_amino_acids_raw = int(num_amino_acids_raw)
        mol_weight_raw = float(mol_weight_raw)

        mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
        length_df = length_df.append({'Uniprot ID': uniprot_id, 'Number of amino acids': num_amino_acids_raw}, ignore_index=True)

        # sleep(1)  # sleep for 1 second
    except Exception as e:
        print(f"Error processing {uniprot_id}: {e}")
        continue

# Merge the two DataFrames
result = pd.merge(mass_df, length_df, on='Uniprot ID')

# Print the resulting DataFrame
print(result)

/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  length_df = length_df.append({'Uniprot ID': uniprot_id, 'Number of amino acids': num_amino_acids_raw}, ignore_index=True)
/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
/var/folders/b

/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  length_df = length_df.append({'Uniprot ID': uniprot_id, 'Number of amino acids': num_amino_acids_raw}, ignore_index=True)
/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
/var/folders/b

/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  length_df = length_df.append({'Uniprot ID': uniprot_id, 'Number of amino acids': num_amino_acids_raw}, ignore_index=True)
/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
/var/folders/b

/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  length_df = length_df.append({'Uniprot ID': uniprot_id, 'Number of amino acids': num_amino_acids_raw}, ignore_index=True)
/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
/var/folders/b

/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  length_df = length_df.append({'Uniprot ID': uniprot_id, 'Number of amino acids': num_amino_acids_raw}, ignore_index=True)
/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
/var/folders/b

/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  length_df = length_df.append({'Uniprot ID': uniprot_id, 'Number of amino acids': num_amino_acids_raw}, ignore_index=True)
/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
/var/folders/b

/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  length_df = length_df.append({'Uniprot ID': uniprot_id, 'Number of amino acids': num_amino_acids_raw}, ignore_index=True)
/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
/var/folders/b

/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  length_df = length_df.append({'Uniprot ID': uniprot_id, 'Number of amino acids': num_amino_acids_raw}, ignore_index=True)
/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
/var/folders/b

   Uniprot ID  Molecular Weight Number of amino acids
0    Q9UHX1-6          55399.46                   513
1    O94925-3          65460.03                   598
2    P06753-2          29032.66                   248
3    P20810-5          81168.58                   756
4    Q13642-1          31894.76                   280
..        ...               ...                   ...
95   Q92851-6          51784.57                   455
96   P14618-2          58062.07                   531
97   Q9Y5K5-4          37352.68                   326
98   Q05682-4          62663.34                   538
99   Q9H1X3-3          16499.04                   153

[100 rows x 3 columns]


/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mass_df = mass_df.append({'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw}, ignore_index=True)
/var/folders/bt/n98c3tq15s3d6866b0l_z_400000gn/T/ipykernel_876/3843952580.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  length_df = length_df.append({'Uniprot ID': uniprot_id, 'Number of amino acids': num_amino_acids_raw}, ignore_index=True)


# Optimising the code to obtain more UniProt IDs, molecular weight, and number of amino acids at once. 

In [ ]:
#To optimize the code and make it run faster, you can use a technique called parallelization. This involves splitting 
#the task into smaller sub-tasks and running them simultaneously on multiple processor cores or machines. 
#This can greatly reduce the total time needed to complete the task. 

In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from time import sleep

def get_fasta_isoform(isoform_id):
    # define the UniProt API endpoint for the protein isoform
    url = f"https://www.uniprot.org/uniprot/{isoform_id}.fasta"

    # send a GET request to the UniProt API endpoint to retrieve the protein sequence
    response = requests.get(url)

    lines = response.text.split('\n')
    fasta_isoform = ''.join(lines[1:])

    return fasta_isoform

def get_protparams(uniprot_id):
    try:
        # Define the URL for the ProtParam tool
        url = "https://web.expasy.org/cgi-bin/protparam/protparam"

        # Define the payload for the GET request
        sequence = get_fasta_isoform(uniprot_id)
        payload = {"sequence": sequence}

        # Submit the GET request and get the response
        response = requests.get(url, params=payload)

        soup = BeautifulSoup(response.content, "html.parser")
        num_amino_acids_raw = soup.find(string="Number of amino acids:").next
        mol_weight_raw = soup.find(string="Molecular weight:").next

        num_amino_acids_raw = int(num_amino_acids_raw)
        mol_weight_raw = float(mol_weight_raw)

        return {'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw, 'Number of amino acids': num_amino_acids_raw}
    except Exception as e:
        print(f"Error processing {uniprot_id}: {e}")
        return None

# Load the UniProt IDs from a CSV file
df = pd.read_csv('/Users/anabelyong/miniconda3/lib/python3.10/site-packages/jupyterlab/Subset750RawfilesCut/CSV_Files_No_Separation/SeparateProteinIDs_No_Separation_ProteinGroups.output/Uniprot_IDs_Isoforms_Raw_Files/filtered_SeparateProteinIDs_Picked_protein_group_no_remap_100Files_withScore_unmerged.csv', nrows=600)

# Use a ThreadPoolExecutor to run the tasks in parallel
with ThreadPoolExecutor() as executor:
    futures = []
    for uniprot_id in df['Protein Uniprot']:
        futures.append(executor.submit(get_protparams, uniprot_id))
        # sleep(1)  # sleep for 1 second

    # Collect the results from the futures
    results = []
    for future in futures:
        result = future.result()
        if result is not None:
            results.append(result)

# Convert the results to a DataFrame
result = pd.DataFrame(results)

# Save the result to a new CSV file
result.to_csv('/Users/anabelyong/miniconda3/lib/python3.10/site-packages/jupyterlab/Subset750RawfilesCut/CSV_Files_No_Separation/SeparateProteinIDs_No_Separation_ProteinGroups.output/Uniprot_IDs_Isoforms_Raw_Files/Uniprot_Mass_test.csv', index=False)

# Print the resulting DataFrame
print(result)


    Uniprot ID  Molecular Weight  Number of amino acids
0     Q9UHX1-6          55399.46                    513
1     O94925-3          65460.03                    598
2     P06753-2          29032.66                    248
3     P20810-5          81168.58                    756
4     Q13642-1          31894.76                    280
..         ...               ...                    ...
400   Q8WWR9-3           9274.42                     84
401   P58743-6          77808.52                    714
402   Q8NB37-2          26784.44                    250
403   Q9BZZ2-3         181733.18                   1701
404   E5RJM6-2          19374.97                    174

[405 rows x 3 columns]


# Testing for multiple csv files to obtain uniprot ID, molecular weight and length of protein through glob module. 

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from time import sleep
import glob

def get_fasta_isoform(isoform_id):
    # define the UniProt API endpoint for the protein isoform
    url = f"https://www.uniprot.org/uniprot/{isoform_id}.fasta"

    # send a GET request to the UniProt API endpoint to retrieve the protein sequence
    response = requests.get(url)

    lines = response.text.split('\n')
    fasta_isoform = ''.join(lines[1:])

    return fasta_isoform

def get_protparams(uniprot_id):
    try:
        # Define the URL for the ProtParam tool
        url = "https://web.expasy.org/cgi-bin/protparam/protparam"

        # Define the payload for the GET request
        sequence = get_fasta_isoform(uniprot_id)
        payload = {"sequence": sequence}

        # Submit the GET request and get the response
        response = requests.get(url, params=payload)

        soup = BeautifulSoup(response.content, "html.parser")
        num_amino_acids_raw = soup.find(string="Number of amino acids:").next
        mol_weight_raw = soup.find(string="Molecular weight:").next

        num_amino_acids_raw = int(num_amino_acids_raw)
        mol_weight_raw = float(mol_weight_raw)

        return {'Uniprot ID': uniprot_id, 'Molecular Weight': mol_weight_raw, 'Number of amino acids': num_amino_acids_raw}
    except Exception as e:
        print(f"Error processing {uniprot_id}: {e}")
        return None

# Get a list of file paths that match a specific pattern
file_paths = glob.glob('/Users/anabelyong/miniconda3/lib/python3.10/site-packages/jupyterlab/Subset750RawfilesCut/CSV_Files_No_Separation/SeparateProteinIDs_No_Separation_ProteinGroups.output/Uniprot_IDs_Isoforms_Raw_Files/*.csv')

# Create an empty list to store the results
results = []

# Loop through the file paths and extract the UniProt IDs
for file_path in file_paths:
    df = pd.read_csv(file_path)
    with ThreadPoolExecutor() as executor:
        futures = []
        for uniprot_id in df['Protein Uniprot']:
            futures.append(executor.submit(get_protparams, uniprot_id))
            # sleep(1)  # sleep for 1 second

        # Collect the results from the futures
        for future in futures:
            result = future.result()
            if result is not None:
                results.append(result)

# Convert the results to a DataFrame
result = pd.DataFrame(results)

# Save the result to a new CSV file
result.to_csv('/Users/anabelyong/miniconda3/lib/python3.10/site-packages/jupyterlab/Subset750RawfilesCut/CSV_Files_No_Separation/SeparateProteinIDs_No_Separation_ProteinGroups.output/Uniprot_IDs_Isoforms_Raw_Files/Uniprot_Mass_test.csv', index=False)

# Print the resulting DataFrame
print(result)
